# Proyecto – Detección de Comunidades

Esta guía proporciona una estructura para el estudio del análisis de comunidades en redes.


* Asignatura: Base de datos 3
* Profesor: Ana Aguilera Faraco
* Ayudante: Fernanda Fuentes
* Fecha: 7 de noviembre del 2025

## Estructura del Estudio

1. Descripción y Preparación del Dataset
2. Análisis de Estructura de Red
3. Métodos de Detección de Comunidades
4. Visualización de Comunidades
5. Métricas y Evaluación de Comunidades
6. Análisis Temporal de Comunidades

# 1. Descripción y Preparación del Dataset

## 1.1 Tipos de Redes para Análisis
* Redes sociales
* Redes biológicas
* Redes de información
* Redes tecnológicas

In [ ]:
# Funciones de utilidad para manejo de tipos de redes
import networkx as nx

def identify_network_type(G):
    """Identifica características básicas de la red"""
    info = {
        "num_nodes": G.number_of_nodes(),
        "num_edges": G.number_of_edges(),
        "is_directed": G.is_directed(),
        "is_weighted": any(isinstance(w, (int, float)) 
                          for _, _, w in G.edges(data="weight", default=1)),
        "has_self_loops": any(G.has_edge(n, n) for n in G.nodes()),
        "density": nx.density(G),
        "avg_degree": sum(dict(G.degree()).values()) / G.number_of_nodes()
    }
    return info

def print_network_info(info):
    """Imprime información de la red en formato legible"""
    print("Características de la Red:")
    print(f"- Nodos: {info['num_nodes']}")
    print(f"- Enlaces: {info['num_edges']}")
    print(f"- Tipo: {'Dirigida' if info['is_directed'] else 'No dirigida'}")
    print(f"- Enlaces con peso: {'Sí' if info['is_weighted'] else 'No'}")
    print(f"- Auto-enlaces: {'Sí' if info['has_self_loops'] else 'No'}")
    print(f"- Densidad: {info['density']:.4f}")
    print(f"- Grado promedio: {info['avg_degree']:.2f}")

## 1.2 Requisitos de los Datos
* Formato de datos de entrada
  - Archivos de texto plano (.txt, .csv)
  - Listas de enlaces (edge lists)
  - Matrices de adyacencia
* Estructuras de grafos
  - Dirigidos vs No dirigidos
  - Con peso vs Sin peso
  - Atributos de nodos y enlaces
* Consideraciones de formato
  - Separadores (espacios, comas, tabs)
  - Encabezados y metadatos
  - Codificación de caracteres

In [ ]:
# Funciones para validación y limpieza de datos
import pandas as pd
import numpy as np

def validate_edge_list(file_path, sep=None, header=None):
    """Valida formato y contenido de una lista de enlaces"""
    try:
        # Intentar leer el archivo
        df = pd.read_csv(file_path, sep=sep, header=header, error_bad_lines=False)
        
        # Verificar formato básico
        if df.shape[1] < 2:
            return False, "El archivo debe tener al menos 2 columnas (origen, destino)"
        
        # Verificar tipos de datos
        if not (df.iloc[:, 0].dtype in ['int64', 'object'] and 
                df.iloc[:, 1].dtype in ['int64', 'object']):
            return False, "Los identificadores deben ser números o texto"
        
        # Verificar duplicados
        duplicates = df.duplicated().sum()
        if duplicates > 0:
            print(f"Advertencia: {duplicates} enlaces duplicados encontrados")
        
        return True, f"Archivo válido: {df.shape[0]} enlaces, {df.nunique().iloc[0]} nodos origen, {df.nunique().iloc[1]} nodos destino"
    
    except Exception as e:
        return False, f"Error al leer el archivo: {str(e)}"

def clean_edge_list(file_path, sep=None, header=None):
    """Limpia y normaliza una lista de enlaces"""
    try:
        df = pd.read_csv(file_path, sep=sep, header=header)
        
        # Eliminar duplicados
        df_clean = df.drop_duplicates()
        
        # Eliminar auto-enlaces si se desea
        df_clean = df_clean[df_clean.iloc[:, 0] != df_clean.iloc[:, 1]]
        
        # Convertir a identificadores numéricos si son strings
        if df_clean.iloc[:, 0].dtype == 'object':
            nodes = pd.concat([df_clean.iloc[:, 0], df_clean.iloc[:, 1]]).unique()
            node_map = {node: i for i, node in enumerate(nodes)}
            df_clean.iloc[:, 0] = df_clean.iloc[:, 0].map(node_map)
            df_clean.iloc[:, 1] = df_clean.iloc[:, 1].map(node_map)
        
        return df_clean, "Limpieza completada exitosamente"
    
    except Exception as e:
        return None, f"Error en la limpieza: {str(e)}"

## 1.3 Preparación de Datos
* Limpieza de datos
  - Eliminación de duplicados
  - Manejo de valores faltantes
  - Corrección de formatos
* Normalización
  - Estandarización de identificadores
  - Conversión de tipos de datos
  - Unificación de formatos
* Validación
  - Verificación de integridad
  - Control de consistencia
  - Detección de anomalías

**Nota**: En esta sección implementaremos:
* Lectura y validación de datos
* Limpieza y normalización
* Creación de estructura de grafo
* Verificación de integridad

## 1.4 Estrategias de Muestreo
* Muestreo de nodos
  - Selección aleatoria
  - Muestreo estratificado
  - Basado en grado
* Muestreo de enlaces
  - Selección aleatoria de enlaces
  - Preservación de comunidades
  - Muestreo por inducción
* Control de calidad
  - Tamaño de muestra
  - Preservación de propiedades
  - Validación estadística

**Nota**: En esta sección implementaremos:
* Técnicas de muestreo
* Validación de la muestra
* Análisis comparativo
* Almacenamiento eficiente

In [ ]:
# Funciones avanzadas de muestreo
import random
import networkx as nx

def random_node_sample(G, sample_size, seed=None):
    """Muestreo aleatorio de nodos"""
    if seed:
        random.seed(seed)
    nodes = list(G.nodes())
    sampled_nodes = random.sample(nodes, min(sample_size, len(nodes)))
    return G.subgraph(sampled_nodes).copy()

def degree_based_sample(G, sample_size, method='top', seed=None):
    """Muestreo basado en grado (top/bottom/random weighted)"""
    if seed:
        random.seed(seed)
    
    degrees = dict(G.degree())
    if method == 'top':
        # Tomar los nodos de mayor grado
        nodes = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
        sampled_nodes = [n for n, _ in nodes[:sample_size]]
    elif method == 'bottom':
        # Tomar los nodos de menor grado
        nodes = sorted(degrees.items(), key=lambda x: x[1])
        sampled_nodes = [n for n, _ in nodes[:sample_size]]
    else:
        # Muestreo ponderado por grado
        total_degree = sum(degrees.values())
        weights = [d/total_degree for d in degrees.values()]
        sampled_nodes = random.choices(list(degrees.keys()), 
                                     weights=weights, 
                                     k=min(sample_size, len(G)))
    
    return G.subgraph(sampled_nodes).copy()

def edge_based_sample(G, sample_size, method='random', seed=None):
    """Muestreo basado en enlaces"""
    if seed:
        random.seed(seed)
    
    edges = list(G.edges())
    if method == 'random':
        # Muestreo aleatorio de enlaces
        sampled_edges = random.sample(edges, min(sample_size, len(edges)))
    else:
        # Muestreo por inducción (preserva estructura local)
        sampled_edges = []
        current_edges = set(random.sample(edges, 1))
        while len(sampled_edges) < sample_size and current_edges:
            edge = current_edges.pop()
            sampled_edges.append(edge)
            # Agregar enlaces adyacentes
            u, v = edge
            current_edges.update(set(G.edges(u)) | set(G.edges(v)) - set(sampled_edges))
    
    # Construir subgrafo con los enlaces muestreados
    H = nx.Graph()
    H.add_edges_from(sampled_edges)
    return H

def validate_sample(G_original, G_sample, metrics=None):
    """Valida la calidad de la muestra comparando métricas"""
    if metrics is None:
        metrics = ['density', 'clustering', 'degree_distribution']
    
    results = {}
    
    if 'density' in metrics:
        results['density'] = {
            'original': nx.density(G_original),
            'sample': nx.density(G_sample)
        }
    
    if 'clustering' in metrics:
        results['clustering'] = {
            'original': nx.average_clustering(G_original),
            'sample': nx.average_clustering(G_sample)
        }
    
    if 'degree_distribution' in metrics:
        orig_degrees = [d for _, d in G_original.degree()]
        sample_degrees = [d for _, d in G_sample.degree()]
        results['avg_degree'] = {
            'original': sum(orig_degrees) / len(orig_degrees),
            'sample': sum(sample_degrees) / len(sample_degrees)
        }
    
    return results

In [ ]:
# Carga y creación de muestra para edgelist grande
import os
import math
import networkx as nx

DATAFILE = 'wiki-topcats.txt'  # ajustar ruta si procede
SAMPLE_PATH = 'wiki-topcats-sample.txt'

def file_size_mb(path):
    try:
        return os.path.getsize(path) / (1024*1024)
    except Exception:
        return None

def build_sample(path, max_lines=200000, save_sample=False):
    G = nx.Graph()
    if not os.path.exists(path):
        print('Archivo no encontrado:', path)
        return G
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for i, line in enumerate(f):
            if max_lines and i >= max_lines:
                break
            parts = line.strip().split()
            if len(parts) < 2:
                continue
            u, v = parts[0], parts[1]
            G.add_edge(u, v)
    if save_sample:
        with open(SAMPLE_PATH, 'w', encoding='utf-8') as out:
            for u, v in G.edges():
                out.write(f"{u} {v}\n")
        print('Muestra guardada en', SAMPLE_PATH)
    return G

# Selección automática: si el archivo es muy grande, usar muestra
size = file_size_mb(DATAFILE)
if size is None:
    print('No se encontró', DATAFILE, 'en el directorio actual. Ajusta DATAFILE si es necesario.')
    G = nx.Graph()
else:
    print(f'File size: {size:.1f} MB')
    if size > 150:
        print('Archivo grande detectado. Construyendo muestra de 200k líneas...')
        G = build_sample(DATAFILE, max_lines=200000, save_sample=False)
    else:
        print('Archivo moderado/pequeño. Construyendo grafo completo...')
        G = build_sample(DATAFILE, max_lines=None, save_sample=False)

print('Grafo construido: Nodos=', G.number_of_nodes(), 'Aristas=', G.number_of_edges())

# 2. Análisis de Estructura de Red

## 2.1 Métricas Básicas
* Densidad de la red
* Distribución de grado
* Coeficiente de clustering
* Longitud de camino promedio

## 2.2 Medidas de Centralidad
* Centralidad de grado
* Centralidad de intermediación
* Centralidad de cercanía
* Centralidad de eigenvector

## 2.3 Propiedades Estructurales
* Componentes conectados
* Puentes y puntos de articulación
* Triángulos y coeficientes locales
* Patrones de conectividad

**Nota**: En esta sección implementaremos:
* Cálculo de métricas básicas
* Análisis de centralidad
* Visualización de distribuciones
* Identificación de estructuras clave

In [ ]:
# Métricas básicas y centralidad (muestra)
import statistics
from collections import Counter

if 'G' not in globals() or G is None or G.number_of_nodes() == 0:
    print('Ejecuta la celda de carga de datos primero.')
else:
    n = G.number_of_nodes(); m = G.number_of_edges()
    print(f'Nodos={n} Aristas={m}')

    # Grado
    degrees = [d for _, d in G.degree()]
    print('Grado: min', min(degrees), 'max', max(degrees), 'mean', round(statistics.mean(degrees), 2))

    # Densidad
    print('Densidad:', nx.density(G))

    # Clustering
    try:
        clustering_vals = nx.clustering(G)
        print('Clustering medio:', round(statistics.mean(clustering_vals.values()), 4))
    except Exception:
        print('Error calculando clustering (grafo dirigido o muy grande)')

    # Componentes
    comps = sorted(nx.connected_components(G), key=len, reverse=True)
    print('Número de componentes:', len(comps))
    print('Tamaño de la componente gigante:', len(comps[0]) if comps else 0)

    # Triángulos (local)
    try:
        tri = nx.triangles(G)
        print('Número total de triángulos (sum/3):', sum(tri.values()) // 3)
    except Exception:
        pass

    # Centralidades (rápido: grado; aproximado: betweenness con k)
    deg_cent = nx.degree_centrality(G)
    top_deg = sorted(deg_cent.items(), key=lambda x: x[1], reverse=True)[:10]
    print('\nTop 10 nodos por centralidad de grado:')
    for node, val in top_deg:
        print(node, round(val, 4))

    try:
        # k se ajusta para aproximar si el grafo es grande
        k = 200 if n > 5000 else None
        bet = nx.betweenness_centrality(G, k=k)
        top_bet = sorted(bet.items(), key=lambda x: x[1], reverse=True)[:10]
        print('\nTop 10 nodos por betweenness (aprox):')
        for node, val in top_bet:
            print(node, round(val, 4))
    except Exception as e:
        print('Betweenness skipped or failed:', e)

    # Histograma de grado si matplotlib disponible
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(6, 4))
        plt.hist(degrees, bins=50, color='#3c8dbc')
        plt.xlabel('Grado')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de grado (muestra)')
        plt.show()
    except Exception as e:
        print('No se pudo graficar distribución de grado:', e)

# 3. Métodos de Detección de Comunidades

## 3.1 Algoritmos Fundamentales
* Método de Louvain
* Algoritmo de Girvan-Newman
* Propagación de Etiquetas
* Optimización de Modularidad

## 3.2 Características de Algoritmos
* Complejidad computacional
* Escalabilidad
* Resolución límite
* Ventajas y limitaciones

## 3.3 Selección de Método
* Criterios de selección
* Tamaño de la red
* Tipo de comunidades esperadas
* Recursos computacionales

**Nota**: En esta sección implementaremos:
* Algoritmos de detección
* Comparación de resultados
* Análisis de rendimiento
* Visualización de comunidades

In [ ]:
# Detección de comunidades: Louvain, Label Propagation y demo Girvan-Newman
from collections import Counter

if 'G' not in globals() or G is None or G.number_of_nodes() == 0:
    print('Ejecuta la celda de carga de datos primero.')
else:
    partition = None
    # Louvain (recomendado)
    try:
        import community as community_louvain
        partition = community_louvain.best_partition(G)
        counts = Counter(partition.values())
        print('Comunidades detectadas (Louvain):', len(counts))
        print('Top 5 comunidades (Louvain):', counts.most_common(5))
    except Exception as e:
        print('python-louvain no disponible o falló:', e)
        partition = None

    # Label Propagation (rápido, sin dependencias externas)
    try:
        from networkx.algorithms.community import label_propagation
        lp_comms = list(label_propagation.label_propagation_communities(G))
        print('Comunidades (Label Propagation) detectadas:', len(lp_comms))
    except Exception as e:
        print('Label Propagation falló:', e)

    # Girvan-Newman demo en subgrafo pequeño (solo ilustrativo)
    try:
        from networkx.algorithms import community as nx_comm
        nodes_small = list(sorted(G.nodes(), key=lambda n: G.degree(n), reverse=True))[:300]
        G_small = G.subgraph(nodes_small).copy()
        comp_gen = nx_comm.girvan_newman(G_small)
        top = tuple(sorted(c) for c in next(comp_gen))
        print('Demo Girvan-Newman (subgrafo 300 nodos): comunidades en primer corte =', len(top))
    except Exception as e:
        print('Girvan-Newman demo falló:', e)

    # leave partition in globals for later cells
    if partition:
        print('Partition dictionary available as `partition` (node -> community)')
    else:
        print('No partition available from Louvain; consider installing python-louvain if deseas esa partición.')

# 4. Visualización de Comunidades

Técnicas efectivas para visualizar y comprender las estructuras comunitarias:

## 4.1 Técnicas de Visualización
* Layouts de fuerza dirigida
* Visualización jerárquica
* Mapas de calor
* Representaciones circulares

## 4.2 Estrategias de Diseño
* Codificación por colores
* Agrupamiento visual
* Escalado de nodos
* Filtrado interactivo

## 4.3 Interpretación Visual
* Patrones estructurales
* Roles de nodos
* Conexiones entre comunidades
* Anomalías y casos especiales

**Nota**: En esta sección implementaremos:
* Creación de visualizaciones
* Personalización de layouts
* Interactividad
* Exportación de gráficos

In [ ]:
# Visualización por comunidad (usa `partition` si está disponible)
import matplotlib.pyplot as plt
import math

if 'G' not in globals() or G is None or G.number_of_nodes() == 0:
    print('Ejecuta la celda de carga de datos primero.')
else:
    # Determinar mapa de comunidades
    if 'partition' in globals() and partition:
        comm_map = partition
    else:
        # fallback: cada componente es su propia 'comunidad'
        comm_map = {n: i for i, comp in enumerate(nx.connected_components(G)) for n in comp}

    # Seleccionar la componente gigante para visualizar
    largest_cc = max(nx.connected_components(G), key=len)
    Gv = G.subgraph(largest_cc).copy()

    # Recalcular comm_map limitado a Gv
    nodes = list(Gv.nodes())
    unique_comms = sorted({comm_map.get(n, 0) for n in nodes})
    cmap = plt.cm.get_cmap('tab20', max(1, len(unique_comms)))
    color_list = [cmap(unique_comms.index(comm_map.get(n, 0))) for n in nodes]

    pos = nx.spring_layout(Gv, seed=42, k=0.1, iterations=50)
    plt.figure(figsize=(10, 8))
    nx.draw_networkx_nodes(Gv, pos, node_size=20, node_color=color_list)
    nx.draw_networkx_edges(Gv, pos, alpha=0.2, width=0.4)
    plt.title('Componente gigante coloreada por comunidad')
    plt.axis('off')
    plt.show()

    # Mostrar tamaños de comunidades en la componente gigante
    size_by_comm = Counter(comm_map.get(n, 0) for n in nodes)
    print('Comunidades en la componente gigante (top 10):', size_by_comm.most_common(10))

C:\Users\Miguel\AppData\Local\Temp\ipykernel_29360\3677995847.py:22: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('tab20', max(1, len(unique_comms)))


# 5. Métricas y Evaluación de Comunidades

Métodos para evaluar la calidad de las comunidades detectadas:

## 5.1 Métricas de Calidad
* Modularidad
* Conductancia
* Cobertura
* Rendimiento

## 5.2 Evaluación Interna
* Densidad intra-comunidad
* Dispersión inter-comunidad
* Cohesión
* Separación

## 5.3 Evaluación Externa
* Ground truth comparación
* Índice de Rand ajustado
* Información mutua normalizada
* F-measure

**Nota**: En esta sección implementaremos:
* Cálculo de métricas
* Comparación de resultados
* Validación de comunidades
* Visualización de métricas

In [ ]:
# Evaluación de comunidades: modularidad, conductancia y métricas externas si hay ground truth
from collections import defaultdict

def conductance(G, community_nodes):
    cut = 0
    volS = 0
    for u in community_nodes:
        volS += G.degree(u)
        for v in G.neighbors(u):
            if v not in community_nodes:
                cut += 1
    vol_rest = sum(dict(G.degree()).values()) - volS
    denom = min(volS, vol_rest) if min(volS, vol_rest) > 0 else 1
    return cut / denom

if 'partition' in globals() and partition:
    try:
        import community as community_louvain
        mod = community_louvain.modularity(partition, G)
        print('Modularidad (Louvain):', round(mod, 4))
    except Exception as e:
        print('No se pudo calcular modularidad (python-louvain):', e)

    # Conductance for top communities
    comms = defaultdict(list)
    for n, c in partition.items():
        comms[c].append(n)
    top = sorted(comms.items(), key=lambda x: len(x[1]), reverse=True)[:3]
    for cid, nodes in top:
        print(f'Conductancia comunidad {cid} (tam={len(nodes)}):', round(conductance(G, set(nodes)), 4))

else:
    print('No hay partición disponible para evaluación. Ejecuta la celda de detección de comunidades.')

# Evaluación externa si hay ground truth en dict `true_labels` (node -> label)
try:
    from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
    if 'true_labels' in globals() and 'partition' in globals() and partition:
        nodes_common = [n for n in G.nodes() if n in true_labels and n in partition]
        y_true = [true_labels[n] for n in nodes_common]
        y_pred = [partition[n] for n in nodes_common]
        print('NMI:', round(normalized_mutual_info_score(y_true, y_pred), 4))
        print('ARI:', round(adjusted_rand_score(y_true, y_pred), 4))
except Exception as e:
    # sklearn puede no estar instalado o no haber ground truth
    pass

## 6.1 Dinámica de Comunidades
* Nacimiento y muerte
* Fusión y división
* Crecimiento y contracción
* Estabilidad y cambio

## 6.2 Métricas Temporales
* Supervivencia de comunidades
* Tasa de cambio
* Persistencia de membresía
* Evolución estructural

## 6.3 Patrones de Evolución
* Ciclos de vida
* Puntos de transición
* Factores de cambio
* Predicción de evolución

**Nota**: En esta sección implementaremos:
* Análisis de series temporales
* Tracking de comunidades
* Visualización de evolución
* Predicción de cambios

---

## Conclusión
Este notebook servirá como guía para el análisis de comunidades en redes, con implementaciones prácticas, ejemplos y visualizaciones.

In [ ]:
# Ejemplo simple de slicing temporal (si hay timestamps en columna 3)
from collections import defaultdict

def slice_windows(path, window_size=86400, max_lines=None):
    windows = defaultdict(list)
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for i, line in enumerate(f):
            if max_lines and i>=max_lines: break
            parts = line.strip().split()
            if len(parts) < 3:
                if len(parts) >= 2:
                    windows[0].append((parts[0], parts[1]))
                continue
            try:
                t = int(parts[2])
            except:
                continue
            win = (t//window_size)*window_size
            windows[win].append((parts[0], parts[1]))
    return windows

print('Si tu edgelist incluye timestamps, usa slice_windows para crear snapshots por ventana.')